# INFO 4271 - Group Project

Issued: June 17, 2025

Due: July 21, 2025

Please submit a link to your code repository (with a branch that does not change anymore after the submission deadline) and your 4-page report via email to carsten.eickhoff@uni-tuebingen.de by the due date. One submission per team.

---

# 1. Web Crawling & Indexing
Crawl the web to discover **English content related to Tübingen**. The crawled content should be stored locally. If interrupted, your crawler should be able to re-start and pick up the crawling process at any time.

In [1]:
%load_ext autoreload
%autoreload 2

from project import Crawler

start_urls = [
    # r"https://duckduckgo.com/html/?q=tübingen%20english",
    "https://en.wikipedia.org/wiki/T%C3%BCbingen",
    # 'https://www.mygermanyvacation.com/best-things-to-do-and-see-in-tubingen-germany/',
    # 'https://www.britannica.com/place/Tubingen-Germany',
    # 'https://www.germany.travel/en/cities-culture/tuebingen.html',
    # 'https://visit-tubingen.co.uk/',
]
crawler = Crawler(urls=start_urls, max_workers=30, use_proxies=False)
# crawler.run()

In [2]:
page = crawler.download_url(start_urls[0])

In [3]:
urls = list(crawler.get_linked_urls(start_urls[0], page.text))
crawler.urls_to_visit = set()
crawler.add_urls_to_visit(urls)

https://upload.wikimedia.org
https://en.wikipedia.org
https://en.wikisource.org
https://viaf.org
http://www.tuebingen.de
https://commons.wikimedia.org
https://geohack.toolforge.org/geohack.php?pagename=T%C3%BCbingen&params=48_31_12_N_09_03_20_E_type:city(91763)_region:DE-BW
https://en.wiktionary.org
https://zh-yue.wikipedia.org
https://search.worldcat.org
https://www.dai-tuebingen.de
https://be-tarask.wikipedia.org
https://developer.wikimedia.org
https://foundation.wikimedia.org
https://en.wikivoyage.org
https://en.m.wikipedia.org/w/index.php?title=T%C3%BCbingen&mobileaction=toggle_view_mobile
https://www.geschichtswerkstatt-tuebingen.de/en/projects/memorial-synagogenplatz#:~:text=The%20T%C3%BCbingen%20Jews%20belonged%20to,,%20lawyers,%20doctors%20and%20bankers.
https://globaltaxnews.ey.com
https://employability.uq.edu.au
https://musicbrainz.org
https://stats.wikimedia.org/#/en.wikipedia.org
https://simple.wikipedia.org
https://en.climate-data.org
https://www.nli.org.il
https://wikimed

2025-07-09 23:11:00,442 INFO: Assumed non english: ['https://www.steiner-verlag.de/Die-Universitaet-Tuebingen-im-Nationalsozialismus/9783515099745', 'https://kopkatalogs.lv/F?func=direct&local_base=lnc10&doc_number=000322839&P_CON_LNG=ENG', 'https://www.dwd.de/DE/leistungen/klimadatendeutschland/mittelwerte/temp_9120_SV_html.html?view=nasPublication&nn=771428', 'https://ro.wikipedia.org/wiki/T%C3%BCbingen', 'https://war.wikipedia.org/wiki/T%C3%BCbingen', 'https://id.worldcat.org/fast/1205517', 'https://af.wikipedia.org/wiki/T%C3%BCbingen', 'https://lb.wikipedia.org/wiki/T%C3%BCbingen', 'https://uk.wikipedia.org/wiki/%D0%A2%D1%8E%D0%B1%D1%96%D0%BD%D0%B3%D0%B5%D0%BD', 'https://www.wikidata.org/wiki/Q3806#identifiers', 'https://eu.wikipedia.org/wiki/T%C3%BCbingen', 'https://stq.wikipedia.org/wiki/T%C3%BCbingen', 'https://sr.wikipedia.org/wiki/%D0%A2%D0%B8%D0%B1%D0%B8%D0%BD%D0%B3%D0%B5%D0%BD', 'https://ko.wikipedia.org/wiki/%ED%8A%80%EB%B9%99%EA%B2%90', 'https://tuebingen.de/', 'https://la

573

In [5]:
filtered = {p for p, d in crawler.visited_pages.items() if d["keywords_found"] and d["is_english"]}
print(len(filtered))
filtered

1


{np.str_('https://www.germany.travel/en/cities-culture/tuebingen.html')}

# 2. Query Processing 
Process a textual query and return the 100 most relevant documents from your index. Please incorporate **at least one retrieval model innovation** that goes beyond BM25 or TF-IDF. Please allow for queries to be entered either individually in an interactive user interface (see also #3 below), or via a batch file containing multiple queries at once. The batch file (see `queries.txt` for an example) will be formatted to have one query per line, listing the query number, and query text as tab-separated entries. An example of the batch file for the first two queries looks like this:

```
1   tübingen attractions
2   food and drinks
```

In [ ]:
#Retrieve documents relevnt to a query. You need (at least) two parameters:
	#query: The user's search query
	#index: The location of the local index storing the discovered documents.
def retrieve(query, index):
    #TODO: Implement me
	pass

# 3. Search Result Presentation
Once you have a result set, we want to return it to the searcher in two ways: a) in an interactive user interface. For this user interface, please think of **at least one innovation** that goes beyond the traditional 10-blue-links interface that most commercial search engines employ. b) as a text file used for batch performance evaluation. The text file should be formatted to produce one ranked result per line, listing the query number, rank position, document URL and relevance score as tab-separated entries. An example of the first three lines of such a text file looks like this:

```
1   1   https://www.tuebingen.de/en/3521.html   0.725
1   2   https://www.komoot.com/guide/355570/castles-in-tuebingen-district   0.671
1   3   https://www.unimuseum.uni-tuebingen.de/en/museum-at-hohentuebingen-castle   0.529
...
1   100 https://www.tuebingen.de/en/3536.html   0.178
2   1   https://www.tuebingen.de/en/3773.html   0.956
2   2   https://www.tuebingen.de/en/4456.html   0.797
...
```

In [ ]:
#TODO: Implement an interactive user interface for part a of this exercise.

#Produce a text file with 100 results per query in the format specified above.
def batch(results):
    #TODO: Implement me.    
    pass

# 4. Performance Evaluation 
We will evaluate the performance of our search systems on the basis of five queries. Two of them are avilable to you now for engineering purposes:
- `tübingen attractions`
- `food and drinks`

The remaining three queries will be given to you during our final session on July 22nd. Please be prepared to run your systems and produce a single result file for all five queries live in class. That means you should aim for processing times of no more than ~1 minute per query. We will ask you to send carsten.eickhoff@uni-tuebingen.de that file.

# Grading
Your final projects will be graded along the following criteria:
- 25% Code correctness and quality (to be delivered on this sheet)
- 25% Report (4 pages, PDF, explanation and justification of your design choices)
- 25% System performance (based on how well your system performs on the 5 queries relative to the other teams in terms of nDCG)
- 15% Creativity and innovativeness of your approach (in particular with respect to your search system #2 and user interface #3 innovations)
- 10% Presentation quality and clarity

# Permissible libraries
You can use any general-puprose ML and NLP libraries such as scipy, numpy, scikit-learn, spacy, nltk, but please stay away from dedicated web crawling or search engine toolkits such as scrapy, whoosh, lucene, terrier, galago and the likes. Pretrained models are fine to use as part of your system, as long as they have not been built/trained for retrieval. 
